In [ ]:
puts `pwd`
puts `ls -l`
puts `head rawdata/disease-therapeuticarea.csv`

In [ ]:
require 'csv'
puts `pwd`

targets = {}

CSV.foreach('./rawdata/disease-therapeuticarea.csv', headers: true) do |row|
  next if row.size < 3

  source = row[0]
  target = row[2] || ''

  target = row[0].match(/([^_]+)/)[1]
  targets[target] = 1

end
puts "Targets"
puts targets.keys

# Process file

Need to process both the source and the target

Source and target need pre-processing to make URLs that are correct for the metadata_functions routines.

*Note:  changed to "append" because Orphanet_3315 has been deprecated and broke the script.  Now restarts at line 14801*


In [ ]:
require_relative '../Lookups/metadata_functions'
require 'csv'

# lookup = NCBO.new
f = File.open('./mappings/disease-therapeuticarea-mappings.csv', 'a')
e = File.open('./mappings/disease-therapeuticarea-errors.txt', 'a')
# f = File.open('./mappings/disease-therapeuticarea-mappings.csv', 'w')
# e = File.open('./mappings/disease-therapeuticarea-errors.txt', 'w')
f.sync = true # Ensure immediate writes
e.sync = true # Ensure immediate writes
#f.write "radboudsource,sourceguid,sourcelabel,radboudtarget,targetguid,targetlabel\n"

seen = {}
count = 1
CSV.foreach('./rawdata/disease-therapeuticarea.csv', headers: true) do |row|
  count = count + 1
  next if count <= 14800
  next if row.size < 3

  # PROCESS SOURCE COLUMN #######################################
  radboudsource = row[0]
  ontology = radboudsource.match(/([^_]+)_/)[1]

  if ontology == "EFO"
    sourceurl = "http://www.ebi.ac.uk/efo/#{radboudsource}"
  elsif ontology == "OTAR"
    sourceurl = "http://www.ebi.ac.uk/efo/#{radboudsource}"
  elsif ontology == "DOID"
    sourceurl = "https://api.disease-ontology.org/v1/terms/#{radboudsource}"
  elsif ontology == "Orphanet"
    sourceurl = "http://www.orpha.net/ORDO/#{radboudsource}"
  else
    sourceurl = "http://purl.obolibrary.org/obo/#{radboudsource}"
  end

  sourcetitle = ontology_annotations(uri: sourceurl)
  # lookup.lookup_title_by_uri(term_uri: url, ontology: ontology)
  if sourcetitle
    warn "#{radboudsource}, #{sourceurl}, #{sourcetitle}"
  else # this shoujld never happen because the raw curie is returned if it fails
    warn "No match for #{source}, #{url}"
    e.write "No match for #{source}, #{url}"
    next
  end
  
# PROCESS TARGET COLUMN #######################################
  radboudtarget = row[2]
  ontology = radboudtarget.match(/([^_]+)_/)[1]

  if ontology == "EFO"
    targeturl = "http://www.ebi.ac.uk/efo/#{radboudtarget}"
  elsif ontology == "OTAR"
    targeturl = "http://www.ebi.ac.uk/efo/#{radboudtarget}"
  elsif ontology == "DOID"
    targeturl = "https://api.disease-ontology.org/v1/terms/#{radboudtarget}"
  elsif ontology == "Orphanet"
    targeturl = "http://www.orpha.net/ORDO/#{radboudtarget}"
  else
    targeturl = "http://purl.obolibrary.org/obo/#{radboudtarget}"
  end

  targettitle = ontology_annotations(uri: targeturl)
  # lookup.lookup_title_by_uri(term_uri: url, ontology: ontology)
  if targettitle
    warn "#{radboudtarget}, #{targeturl}, #{targettitle}"
  else # this shoujld never happen because the raw curie is returned if it fails
    warn "No match for #{radboudtarget}, #{targeturl}"
    e.write "No match for #{radboudtarget}, #{targeturl}"
    next  
  end
  
  

  f.write CSV.generate_line([radboudsource, sourceurl, sourcetitle, radboudtarget, targeturl, targettitle])

end

f.close
e.close